#  installing and importing packages

In [67]:
pip install betterreads

Note: you may need to restart the kernel to use updated packages.


In [146]:
import pandas as pd
import numpy as np
from betterreads import client
from goodreads_data_gather import helper

# Data Gathering

In [147]:
h = helper()

in constructor 


In [171]:
books = pd.read_csv('../data/books.csv')
print( f"shape of the books dataframe is {books.shape}")
ratings = pd.read_csv("../data/ratings.csv")
print( f"shape of the books dataframe is {ratings.shape}")
book_tags = pd.read_csv("../data/book_tags.csv")
print( f"shape of the books dataframe is {book_tags.shape}")
tags = pd.read_csv("../data/tags.csv")
print( f"shape of the books dataframe is {tags.shape}")
books.head(1)

shape of the books dataframe is (10000, 23)
shape of the books dataframe is (5976479, 3)
shape of the books dataframe is (999912, 3)
shape of the books dataframe is (34252, 2)


,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...


In [172]:
tags.head(1)

,tag_id,tag_name
0,0,-


In [173]:
book_tags.head(1)

,goodreads_book_id,tag_id,count
0,1,30574,167697


In [174]:
ratings.head(1)

,user_id,book_id,rating
0,1,258,5


In [175]:
books.head(1)

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...


In [ ]:
### Combining booktag ids with the actual names

In [ ]:
pd.merge(left=book_tags,right =tags, how ="left", on ='tag_id')
book_tags.to_csv("../data/book_tags_combined",index =False)

In [176]:
books.isna().sum()

book_id                         0
goodreads_book_id               0
best_book_id                    0
work_id                         0
books_count                     0
isbn                          700
isbn13                        585
authors                         0
original_publication_year      21
original_title                585
title                           0
language_code                1084
average_rating                  0
ratings_count                   0
work_ratings_count              0
work_text_reviews_count         0
ratings_1                       0
ratings_2                       0
ratings_3                       0
ratings_4                       0
ratings_5                       0
image_url                       0
small_image_url                 0
dtype: int64

585 books that are missing orig title, replace them with title
book = gc.book(3)
book.original_title

In [177]:
len(books), len(books[books['original_title'].isna()]),len(books[books['original_title'] != books ['title']])

(10000, 585, 6015)

In [178]:
# Title seems to be including original title in all these cases
books[books['original_title'] != books ['title']][['goodreads_book_id','title','original_title']]

,goodreads_book_id,title,original_title
0,2767052,"The Hunger Games (The Hunger Games, #1)",The Hunger Games
1,3,Harry Potter and the Sorcerer's Stone (Harry P...,Harry Potter and the Philosopher's Stone
2,41865,"Twilight (Twilight, #1)",Twilight
6,5907,The Hobbit,The Hobbit or There and Back Again
8,960,"Angels & Demons (Robert Langdon, #1)",Angels & Demons
...,...,...,...
9988,13489518,"Out of the Dark (The Grey Wolves, #4)",Out of the Dark
9991,13616278,"The Red Knight (The Traitor Son Cycle, #1)",The Red Knight
9992,4936457,The Time Traveller's Guide to Medieval England...,The Time-Traveller's Guide to Medieval England...
9995,7130616,"Bayou Moon (The Edge, #2)",Bayou Moon


In [179]:
#Drop columns that are not relevant for the recommender
books = books.drop(['best_book_id','work_id','isbn13','isbn','original_title',
                    'books_count','work_ratings_count','work_text_reviews_count',
                    'image_url','small_image_url'], axis =1)

In [180]:
books['description'] =np.nan
books['num_pages'] =0
books['e_book'] =False

In [181]:
books =h.fill_missing_columns(books)

array([    1,     2,     3, ...,  9998,  9999, 10000])

In [ ]:
def fill_missing_columns(books)
    missing_desc_books = h.get_book_id_list_for_missing_column(books,'description')
    for i in missing_desc_books:
        goodreads_book_id = books.loc[books['book_id'] == i,'goodreads_book_id'].values[0]
        try:
            book_obj= h.get_book(goodreads_book_id)
            if(i % 100 ==1):
                print( f"{i},{book_obj}")
            books.loc[books['book_id'] == i,'description']= book_obj.description
            try:
                books.loc[books['book_id'] == i,'num_pages']=  book_obj.num_pages
            except:
                books.loc[books['book_id'] == i,'num_pages']=  0

            books.loc[books['book_id'] == i,'is_ebook']=book_obj.is_ebook  
        except:
            books.loc[books['book_id'] == i,'num_pages']=  0
            books.loc[books['book_id'] == i,'description'] =""
            books.loc[books['book_id'] == i,'is_ebook'] =False
            print(f"failed to parse {i}")
    return books      

1,The Hunger Games (The Hunger Games, #1)
101,Me Talk Pretty One Day
201,The Lost Symbol (Robert Langdon, #3)
301,Heart of Darkness
401,A Farewell to Arms
501,The No. 1 Ladies' Detective Agency (No. 1 Ladies' Detective Agency #1)
601,The Tale of Peter Rabbit
701,Mrs. Dalloway
801,This is Where I Leave You
901,Year of Wonders
1001,The Dinner
1101,Change of Heart
1201,Dead Reckoning (Sookie Stackhouse, #11)
1301,The Broker
1401,The Artist's Way
failed to parse 1475
1501,The Queen's Fool (The Plantagenet and Tudor Novels, #12)
1601,Emotional Intelligence: Why It Can Matter More Than IQ
1701,I Hope They Serve Beer in Hell (Tucker Max, #1)
1801,The Sword of Shannara (The Original Shannara Trilogy, #1)
1901,Shadowfever (Fever, #5)
2001,Harry Potter: Film Wizardry
2101,The Harry Potter Collection 1-4 (Harry Potter, #1-4)
failed to parse 2139
failed to parse 2151
2201,Wings (Wings, #1)
2301,The Ask and the Answer (Chaos Walking, #2)
2401,The Stranger
2501,Soul Music (Discworld, #16; Death, #3)

In [ ]:
len(books[books['description'].isna()])

### Collecting  recent books  based on Denver DSI classmates

In [96]:
collect_books = pd.DataFrame()
topics =['python','of Thrones','Patrick Rothfuss','A Song of Ice and Fire','Inspector Rebus',
      'Data Science', 'mechine learning','Data Visualization','Robert T. Kiyosaki','Brené Brown',
      'weapons of math destruction','Girl, Wash Your Face','recommender systems']

In [97]:
for topic in topics :
    df2 =h.get_books(topic)
    books = pd.concat([books,df2])
books.reset_index(inplace=True, drop=True)     

/Users/mybiyani/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [98]:
books.shape

(10273, 19)

In [135]:
books.isna().sum()

authors                        0
average_rating                 0
book_id                      273
description                  106
e_book                       273
goodreads_book_id              0
is_ebook                       0
language_code                  0
num_pages                      0
original_publication_year      0
ratings_1                    273
ratings_2                    273
ratings_3                    273
ratings_4                    273
ratings_5                    273
ratings_count                  0
title                          0
publication_year               0
publication_timeframe          0
dtype: int64

In [136]:
#books = books.drop(['isbn','original_title'], axis =1)

In [ ]:
###### Are the books unique ?

we have books unique isbn numbers  but  they have similar titles.
I am little wary of this data set now. It might  be a good  for proof of concept but  we may need to get cleaner data

In [139]:
#De-duplicating the  records as we only care about title and content of the book , 
books_with_same_title = (books['title'].value_counts()  )
print(f"duplicate book count = {len(books_with_same_title.index[books_with_same_title >1])}")

dup_books =books_with_same_title.index[books_with_same_title >1]
dup_book_df = books[books['title'].isin(dup_books)]

books = books.groupby('title').agg(
                             {
                              'book_id':'first',
                              'goodreads_book_id':'first',
                              'authors':'first', 
                              'description' :'first',
                              'original_publication_year': 'first',
                              'language_code' :'first',
                              'average_rating':'mean',
                              'ratings_count': 'sum',                              
                              'num_pages':'first',
                              'is_ebook':'first',
                              'ratings_count':'first'
                              }).reset_index()

duplicate book count = 163


In [140]:
books['language_code'] = books['language_code'].apply(lambda x : 'eng' if x in  ['eng','en-US','en-GB','en-CA'] else 'other')
books['language_code'].value_counts()

eng      8757
other    1319
Name: language_code, dtype: int64

In [141]:
books['original_publication_year'] = books['original_publication_year'].fillna(0).astype('int')
books['original_publication_year'].describe()

count    10076.000000
mean      1957.329694
std        259.545660
min          0.000000
25%       1989.000000
50%       2004.000000
75%       2011.000000
max       2017.000000
Name: original_publication_year, dtype: float64

In [142]:
books.loc[books['original_publication_year'] <0,'original_publication_year'] =0

In [134]:
def publication_year_categorize(original_publication_year):
    if original_publication_year >=2015 :
        return('2015-2020')
    elif original_publication_year >= 2010:
        return('2010-2014')
    elif original_publication_year >= 2000:
        return('2000-2013')
    elif original_publication_year >= 1990:
        return('1990-2000')
    elif original_publication_year >= 1950:
        return('1950-1990')
    elif original_publication_year >= 1:
        return('<1950')
    else:
         return('unknown')
books['publication_timeframe'] =books['original_publication_year'].map(publication_year_categorize)
books['publication_timeframe'].value_counts()

2000-2013    3121
2010-2014    2552
1950-1990    1586
1990-2000    1360
<1950         814
2015-2020     515
unknown       325
Name: publication_timeframe, dtype: int64

# Data Cleaning

###### How many of the  columns are missing data ?

In [143]:
books.isna().sum()

title                          0
book_id                      112
goodreads_book_id              0
authors                        0
description                  105
original_publication_year      0
language_code                  0
average_rating                 0
ratings_count                  0
num_pages                      0
is_ebook                       0
dtype: int64

In [ ]:
###### If the book description was empty ,  adding a string so that  Vecotrizer can  work with the data

,goodreads_book_id,tag_id,count,tag_name
0,1,30574,167697,to-read
1,1,11305,37174,fantasy
2,1,11557,34173,favorites
3,1,8717,12986,currently-reading
4,1,33114,12716,young-adult
...,...,...,...,...
999907,33288638,21303,7,neighbors
999908,33288638,17271,7,kindleunlimited
999909,33288638,1126,7,5-star-reads
999910,33288638,11478,7,fave-author


In [144]:
books['description'] = books['description'].fillna('-')

In [ ]:
###### Consolidate all Audio tags under same category
def consolidate_tags(tag_lst,target_tag):
    target_tag_id = book_tags[book_tags['tag_name']==target_tag]['tag_id'].values[0]  
    l= book_tags[(book_tags['tag_name'].isin(tag_lst))]['tag_name'].to_list()
    print(f"replacing {len(l)} tags with {target_tag}, tag_id= {target_tag_id}")
    book_tags['tag_name']= book_tags['tag_name'].apply(lambda x : target_tag if x in tag_lst else x)
    book_tags.loc[book_tags['tag_name'] ==target_tag,'tag_id']= target_tag_id 

In [ ]:
##### Consolidate all childrens books under same category
#tag_lst= book_tags[(book_tags['tag_name'].str.contains('children')) ]['tag_name'].to_list()
#consolidate_tags(list(set(tag_lst)),'childrens')

#tag_lst= ['audio','audiobooks','audiobook','audible','audio-book','audio-books'] +(book_tags[(book_tags['tag_name'].str.contains('audiobook'))]['tag_name'].to_list())
#consolidate_tags(set(tag_lst),'audiobook')

# tag_lst = book_tags[(book_tags['tag_name'].str.contains('history'))]['tag_name'].to_list()
# consolidate_tags(set(['war'] +tag_lst),'history')

#suspense
# tag_lst= book_tags[(book_tags['tag_name'].str.contains('mystery'))]['tag_name'].to_list()
# consolidate_tags(tag_lst,'mystery-thriller')


# tag_lst=set(book_tags[(book_tags['tag_name'].str.contains('novel'))]['tag_name'].to_list())
# consolidate_tags(tag_lst,'novels')

# consolidate_tags(['teen','juvenile','youth','new-adult'],'young-adult')
# consolidate_tags(['general-fiction','speculative-fiction','magic','literary-fiction','contemporary-fiction','modern-fiction','vampires','vampire','fiction-to-read'],'fiction')
# consolidate_tags(['paranormal','paranormal-romance','paranormal-fantasy','fantasy-paranormal'],'supernatural') 
# consolidate_tags(['fantasy-sci-fi,sci-fi-fantasy','science-fiction-fantasy','fantasy-scifi','sci-fi-and-fantasy'],'scifi-fantasy') 
# consolidate_tags(['funny','humour','comedy','humorous'],'humor') 
# consolidate_tags(['20th-century','19th-century','historicals'],'historical') 
# consolidate_tags(['sci-fi','scifi'],'science-fiction') 

#consolidate_tags(['non-fiction','to-read-nonfiction','non-fic','nonfiction-to-read'],'nonfiction') 
#consolidate_tags(['ya-fiction','harry-potter'],'fiction') 

# consolidate_tags(['urban-fantasy','ya-fantasy','high-fantasy','sf-fantasy','guilty-pleasures'],'fantasy')
# consolidate_tags(['thrillers'],'thriller')
# consolidate_tags(['childhood-reads','kids','kids-books','childhood-books','childhood','childhood-favorites'],'childrens')
# consolidate_tags(['listened-to'],'audiobook')
# consolidate_tags(['to-read-non-fiction','non-fiction-to-read'],'nonfiction') 
# consolidate_tags(['contemporary-romance'],'romance')
# consolidate_tags(['urban-fantasy','ya-fantasy'],'fantasy')


# tag_lst=set(book_tags[(book_tags['tag_name'].str.contains('memoir'))]['tag_name'].to_list())
# consolidate_tags(tag_lst,'biography')

# consolidate_tags(['teen-fiction','juvenile-fiction'],'young-adult-fiction')
# consolidate_tags(['classic','literature','literary','classics','american-lit','american-literature','british-literature','chic-lit','kid-lit','chicklit','to-read-classics'],'classic-literature') 
# consolidate_tags(['dystopia'],'dystopian')



# consolidate_tags(['realistic'],'realistic-fiction')
# consolidate_tags(['erotica','erotic'],'adult-romance')
# consolidate_tags(['modern'],'contemporary')
consolidate_tags(['suspense-thriller','thriller-mystery','thriller-suspense','mystery-thrillers','epic-fantasy','mystery-suspense-thriller','mystery-thriller-suspense'],'mystery-thriller')


In [145]:
books.to_csv("../data/books_desc.csv", index=False)


######  references
https://pypi.org/project/Goodreads/

https://betterreads.readthedocs.io/en/latest/

Kaggle data source : https://www.kaggle.com/zygmunt/goodbooks-10k

data set location : https://github.com/zygmuntz/goodbooks-10k

#https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html